In [1]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from scipy import stats
import math
import statistics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize
%matplotlib inline

# Split datasets into train and test datasets


df_main_onehot = pd.read_csv('df_main_onehot.csv')
# df_merged_venue = pd.read_csv('df_merged_venue.csv')
# df_merged_venue['storey_avg'] = df_merged_venue['storey_range'].apply(lambda x: int(x[-2:]))


In [15]:
df_all_parsed = pd.read_csv('df_venues_number.csv')
df_all_parsed

,Unnamed: 0,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,0,serangoon north avenue 3,1.371711,103.871234,21,17,20,3,14,5,0
1,1,simei road,1.344802,103.959862,8,7,7,10,6,2,0
2,2,tampines street 12,1.348070,103.943577,13,13,37,3,17,5,1
3,3,fernvale link,1.389799,103.880644,5,4,6,6,8,3,1
4,4,teban gardens road,1.321139,103.742788,6,8,10,7,6,3,0
...,...,...,...,...,...,...,...,...,...,...,...
583,443,elias road,1.373220,103.940290,4,11,7,6,10,3,0
584,444,holland close,1.307305,103.796033,68,38,60,4,54,25,1
585,445,tanglin halt road,1.300528,103.798360,35,24,26,9,23,11,0
586,447,wellington circle,1.452079,103.820354,29,14,33,6,24,10,0


In [16]:
df_main_onehot

,month,floor_area_sqm,remaining_lease,price_sqm,distance_to_nearest_stations,distance_to_nearest_primary_sch,distance_to_nearest_second_sch,distance_to_nearest_commercial,distance_to_nearest_hawker,distance_to_nearest_malls,...,town_toa payoh,town_woodlands,town_yishun,storey_avg,storey < 10,storey < 15,storey < 20,storey < 30,story > 30,year
0,8,118.0,71,1777.118644,1.137651,0.343632,0.426132,2.307216,0.944998,1.028772,...,0,0,0,3,1,0,0,0,0,2001
1,10,110.0,85,3657.272727,0.821397,0.160104,0.311598,1.455538,3.773975,0.812607,...,0,0,0,12,0,1,0,0,0,2014
2,9,112.0,86,3133.928571,2.232095,0.183945,0.558057,1.445009,1.797718,0.450920,...,0,0,0,3,1,0,0,0,0,2020
3,10,67.0,62,2256.716418,0.421346,0.303219,0.618757,2.386699,0.615444,0.454836,...,0,0,0,9,1,0,0,0,0,2000
4,1,73.0,67,4364.383562,0.775005,0.233534,0.218129,1.800060,0.760363,0.764628,...,0,0,0,9,1,0,0,0,0,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431727,3,101.0,82,2361.386139,0.300402,0.210006,0.336660,1.036933,1.470056,1.014346,...,0,1,0,3,1,0,0,0,0,2005
431728,4,95.0,94,3960.000000,2.070084,0.192740,0.262617,1.609766,1.844287,0.190375,...,0,0,0,15,0,0,0,0,0,2016
431729,1,67.0,68,3814.925373,0.567511,0.187832,0.250725,1.224333,0.723242,0.766860,...,0,0,0,3,1,0,0,0,0,2011
431730,5,123.0,81,4134.146341,0.614204,0.159620,0.288516,1.616832,2.581630,0.682419,...,0,0,0,18,0,0,1,0,0,2013


In [9]:
df_main_onehot = pd.read_csv('df_main_onehot.csv')
df_main_onehot = df_main_onehot.drop(columns=['Unnamed: 0','flat_type','month', 'street_name',
       'flat_model', 'planning_area','distance_to_nearest_primary_sch','distance_to_nearest_second_sch','distance_to_nearest_hawker'])

In [12]:
df_main_onehot.columns

Index(['floor_area_sqm', 'remaining_lease', 'price_sqm',
       'distance_to_nearest_stations', 'distance_to_nearest_commercial',
       'distance_to_nearest_malls', 'toilet_number', 'room_number',
       'flat_type_1 room', 'flat_type_2 room', 'flat_type_3 room',
       'flat_type_4 room', 'flat_type_5 room', 'flat_type_executive',
       'flat_type_multi generation', 'flat_model_2 room',
       'flat_model_adjoined flat', 'flat_model_apartment', 'flat_model_dbss',
       'flat_model_improved', 'flat_model_improved maisonette',
       'flat_model_maisonette', 'flat_model_model a',
       'flat_model_model a maisonette', 'flat_model_model a2',
       'flat_model_multi generation', 'flat_model_new generation',
       'flat_model_premium apartment', 'flat_model_premium apartment loft',
       'flat_model_premium maisonette', 'flat_model_simplified',
       'flat_model_standard', 'flat_model_terrace', 'flat_model_type s1',
       'flat_model_type s2', 'town_ang mo kio', 'town_bedok', 'tow

### df_main_onehot + lasso 

In [14]:
y = df_main_onehot['price_sqm']
X = df_main_onehot.drop(columns = ['price_sqm'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.25, random_state = 888)

# scaler = MinMaxScaler().fit(X_train)
scaler = StandardScaler().fit(X_train)
X_trains = scaler.transform(X_train)
X_tests = scaler.transform(X_test)

lasso = Lasso()
param = {'alpha':[0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 1, 1.5]}

lasso_cv = GridSearchCV(lasso,
                       param,
                       scoring = 'neg_mean_squared_error',
                       cv = 5,
                       verbose = 0)
lasso_cv.fit(X_trains,y_train)

best_alpha = pd.Series(lasso_cv.best_params_)[0]

print(lasso_cv.best_score_)
print(lasso_cv.best_params_)


/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3284636719.0378113, tolerance: 33022952.127027918
  model = cd_fast.enet_coordinate_descent(
/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3272732770.394394, tolerance: 32962727.989448633
  model = cd_fast.enet_coordinate_descent(
/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3288651867.190468, tolerance: 33002043.041918166
  model = cd_fast.enet_coordinate_descent(
/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_desce

-327051.63963414874
{'alpha': 0.01}


/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4097999553.949692, tolerance: 41227085.091925636
  model = cd_fast.enet_coordinate_descent(


In [15]:
lasso = Lasso(alpha = best_alpha, 
              random_state = 888)
lasso = lasso.fit(X_trains, y_train)

/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4097999553.949692, tolerance: 41227085.091925636
  model = cd_fast.enet_coordinate_descent(


In [16]:
X.columns

Index(['floor_area_sqm', 'remaining_lease', 'distance_to_nearest_stations',
       'distance_to_nearest_commercial', 'distance_to_nearest_malls',
       'toilet_number', 'room_number', 'flat_type_1 room', 'flat_type_2 room',
       'flat_type_3 room', 'flat_type_4 room', 'flat_type_5 room',
       'flat_type_executive', 'flat_type_multi generation',
       'flat_model_2 room', 'flat_model_adjoined flat', 'flat_model_apartment',
       'flat_model_dbss', 'flat_model_improved',
       'flat_model_improved maisonette', 'flat_model_maisonette',
       'flat_model_model a', 'flat_model_model a maisonette',
       'flat_model_model a2', 'flat_model_multi generation',
       'flat_model_new generation', 'flat_model_premium apartment',
       'flat_model_premium apartment loft', 'flat_model_premium maisonette',
       'flat_model_simplified', 'flat_model_standard', 'flat_model_terrace',
       'flat_model_type s1', 'flat_model_type s2', 'town_ang mo kio',
       'town_bedok', 'town_bishan', 't

In [17]:
X_trains

array([[-0.27682309,  1.12957491,  0.09230356, ..., -0.1801294 ,
        -0.04372918,  1.02453305],
       [ 0.27893557, -0.62624964,  0.19812948, ..., -0.1801294 ,
        -0.04372918,  1.35429707],
       [ 0.35832966,  0.54430006,  0.75023554, ..., -0.1801294 ,
        -0.04372918,  1.18941506],
       ...,
       [-1.58682564, -1.60170772, -0.64038207, ..., -0.1801294 ,
        -0.04372918,  1.02453305],
       [ 0.95378537,  0.83693749,  0.24257807, ..., -0.1801294 ,
        -0.04372918,  0.20012298],
       [-1.18985517, -1.11397868, -0.8282911 , ..., -0.1801294 ,
        -0.04372918,  1.84894311]])

In [18]:
yhat = lasso.predict(X_trains)
MAE_lasso = round(mean_absolute_error(y_train, yhat),2)
RMSE_lasso = round(math.sqrt(mean_squared_error(y_train, yhat)),2)
R2_lasso = round(r2_score(y_train, yhat),2)


print('-------------- Lasso Regression Model on training set------------')
print('MAE: ' + str(mean_absolute_error(y_train, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_train, yhat))))
print('R^2 Score: ' + str(r2_score(y_train, yhat)))

-------------- Lasso Regression Model on training set------------
MAE: 450.16826939494763
RMSE: 571.7500061993912
R^2 Score: 0.7432530876271427


In [19]:
yhat = lasso.predict(X_tests)
MAE_lasso = round(mean_absolute_error(y_test, yhat),2)
RMSE_lasso = round(math.sqrt(mean_squared_error(y_test, yhat)),2)
R2_lasso = round(r2_score(y_test, yhat),2)
nh bv

-------------- Lasso Regression Model ------------
MAE: 451.16153269510903
RMSE: 573.2498000425496
R^2 Score: 0.7414491437856264


In [27]:
df_main_onehot.astype(float)

,month,floor_area_sqm,remaining_lease,price_sqm,distance_to_nearest_stations,distance_to_nearest_primary_sch,distance_to_nearest_second_sch,distance_to_nearest_commercial,distance_to_nearest_hawker,distance_to_nearest_malls,...,town_toa payoh,town_woodlands,town_yishun,storey_avg,storey < 10,storey < 15,storey < 20,storey < 30,story > 30,year
0,8.0,118.0,71.0,1777.118644,1.137651,0.343632,0.426132,2.307216,0.944998,1.028772,...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2001.0
1,10.0,110.0,85.0,3657.272727,0.821397,0.160104,0.311598,1.455538,3.773975,0.812607,...,0.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,0.0,2014.0
2,9.0,112.0,86.0,3133.928571,2.232095,0.183945,0.558057,1.445009,1.797718,0.450920,...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2020.0
3,10.0,67.0,62.0,2256.716418,0.421346,0.303219,0.618757,2.386699,0.615444,0.454836,...,0.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,2000.0
4,1.0,73.0,67.0,4364.383562,0.775005,0.233534,0.218129,1.800060,0.760363,0.764628,...,0.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431727,3.0,101.0,82.0,2361.386139,0.300402,0.210006,0.336660,1.036933,1.470056,1.014346,...,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2005.0
431728,4.0,95.0,94.0,3960.000000,2.070084,0.192740,0.262617,1.609766,1.844287,0.190375,...,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,2016.0
431729,1.0,67.0,68.0,3814.925373,0.567511,0.187832,0.250725,1.224333,0.723242,0.766860,...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2011.0
431730,5.0,123.0,81.0,4134.146341,0.614204,0.159620,0.288516,1.616832,2.581630,0.682419,...,0.0,0.0,0.0,18.0,0.0,0.0,1.0,0.0,0.0,2013.0


### df_main_onehot + xgboost 

In [2]:
df_main_onehot = pd.read_csv('df_main_onehot.csv')
df_main_onehot = df_main_onehot.drop(columns=['Unnamed: 0','flat_type', 'street_name',
       'flat_model', 'planning_area'])
df_main_onehot['month'] = df_main_onehot['month'].apply(lambda x: int(x[-2:]))
y = df_main_onehot['price_sqm']
X = df_main_onehot.drop(columns = ['price_sqm'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.25, random_state = 888)

# scaler = MinMaxScaler().fit(X_train)
scaler = StandardScaler().fit(X_train)
X_trains = scaler.transform(X_train)
X_tests = scaler.transform(X_test)

In [30]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold

# Convert datasets to XGBDMatrix
# dtrain = xgb.DMatrix(X_trains, label = y_train, feature_names=X.columns)
dtrain = xgb.DMatrix(X_trains, label = y_train)
dtest = xgb.DMatrix(X_tests)

# Define model parameters
xgb_params = {'obj':'reg:squarederror',
              'eval_metric':'rmse',
              'max_depth':6, 
              'eta':0.5,
              'min_child_weight':1,
              'subsample':1,
              'colsample_bytree':1}

# Find model parameters 'num_boost_round' using cv
cv_res = xgb.cv(xgb_params, 
                dtrain, 
                num_boost_round=500, 
                nfold=5, 
                seed=2020, 
                stratified=False,
                early_stopping_rounds=25, 
                verbose_eval=10, 
                show_stdv=True)

# Get best num_boost_round
nrounds = cv_res[cv_res['test-rmse-mean'] == cv_res['test-rmse-mean'].min()].index[0]
cv_res[cv_res['test-rmse-mean'] == cv_res['test-rmse-mean'].min()]

xgb_def = xgb.train(params = xgb_params,
                   num_boost_round = nrounds,
                   dtrain = dtrain)

yhat = xgb_def.predict(dtest)
print('-------------- XGB Model: Baseline Parameters ------------')
print('MAE: ' + str(mean_absolute_error(y_test, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_test, yhat))))
print('R^2 Score: ' + str(r2_score(y_test, yhat)))

[17:43:29] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:43:31] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:43:32] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are 

In [63]:
set(X_train.columns).difference(set(x_test.columns))

{'distance_to_nearest_hawker',
 'distance_to_nearest_primary_sch',
 'distance_to_nearest_second_sch',
 'month'}

In [70]:
x_test = pd.read_csv('test_onehot.csv') 
x_test.columns

Index(['Unnamed: 0', 'month', 'floor_area_sqm', 'remaining_lease',
       'distance_to_nearest_stations', 'distance_to_nearest_primary_sch',
       'distance_to_nearest_second_sch', 'distance_to_nearest_commercial',
       'distance_to_nearest_hawker', 'distance_to_nearest_malls',
       'toilet_number', 'room_number', 'flat_type_1 room', 'flat_type_2 room',
       'flat_type_3 room', 'flat_type_4 room', 'flat_type_5 room',
       'flat_type_executive', 'flat_type_multi generation',
       'flat_model_2 room', 'flat_model_adjoined flat', 'flat_model_apartment',
       'flat_model_dbss', 'flat_model_improved',
       'flat_model_improved maisonette', 'flat_model_maisonette',
       'flat_model_model a', 'flat_model_model a maisonette',
       'flat_model_model a2', 'flat_model_multi generation',
       'flat_model_new generation', 'flat_model_premium apartment',
       'flat_model_premium apartment loft', 'flat_model_premium maisonette',
       'flat_model_simplified', 'flat_model_stand

In [3]:
x_test = pd.read_csv('test_onehot.csv') 
# x_test['month'] = x_test['month'].apply(lambda x: int(x[-2:]))
# x_test = x_test.drop(columns=['Unnamed: 0'])
# x_test = scaler.transform(x_test)
# x_test = xgb.DMatrix(x_test)
# y_test = xgb_def.predict(x_test)
# y_test 
y_test = pd.read_csv('final_re.csv')

In [9]:
area_sqm = x_test['floor_area_sqm']

y_t = y_test['0']
list(y_t), list(area_sqm)
pred = [area_sqm[i] * y_t[i] for i in range(0,len(y_t))]

([2139.557,
  2512.3276,
  1866.081,
  4001.259,
  2798.7139,
  2573.3489,
  3127.8828,
  2339.8633,
  2599.7686,
  3068.0537,
  2263.713,
  3053.6384,
  3701.2517,
  4061.7854,
  1711.1847,
  2160.6584,
  3784.7417,
  1878.2074,
  2945.645,
  1869.7633,
  1859.6356,
  3417.1382,
  1888.009,
  3645.3047,
  1918.607,
  5076.1724,
  1803.6227,
  3039.3079,
  2837.7466,
  2041.5513,
  2414.5332,
  2099.1787,
  3457.5442,
  3775.016,
  4450.7095,
  2046.9578,
  2777.3765,
  2513.342,
  3304.4722,
  1818.3474,
  4539.218,
  4513.233,
  4643.7246,
  2424.4124,
  2500.8408,
  3695.5852,
  2151.297,
  5114.7173,
  2464.6382,
  4628.1934,
  2205.4983,
  5108.2017,
  3404.9648,
  2291.1204,
  3885.2441,
  2320.6692,
  1696.0675,
  3300.7837,
  3302.425,
  2026.9602,
  4215.073,
  4020.38,
  2294.721,
  2054.145,
  2784.5532,
  2475.537,
  3391.4734,
  2512.7263,
  2355.9478,
  4634.2217,
  2397.6921,
  3249.5378,
  4785.224,
  2055.6538,
  3082.053,
  5100.299,
  3268.9712,
  3007.752,
  3597.71

In [14]:
pd.DataFrame(pred).to_csv('final_result.csv')